# Custom loss function using pre-calculated derivatives

Notebook to work through developing a custom loss function as in "JP861 - 003" but calculating derivatives across whole dataset prior to creating datasets to allow significant smoothing to be performed.

This code is largely based on "AI4ER GTC - Slow Earthquake Time Series Forecasting.ipynb"

## Imports

In [1]:
# Set the system directories
import sys
import os
import socket
from utils.paths import MAIN_DIRECTORY

if MAIN_DIRECTORY not in sys.path:
    sys.path.append(MAIN_DIRECTORY)

ROOT_DIRECTORY = "sys.path.append(os.getcwd() + '/..')"
if ROOT_DIRECTORY not in sys.path:
    sys.path.append(ROOT_DIRECTORY)

# Import the standard libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import pickle

# Import local modules - note: dependent on above path being set correctly
from scripts.models.lstm_oneshot_multistep import MultiStepLSTMSingleLayer
from scripts.models.tcn_oneshot_multistep import MultiStepTCN
from utils.data_preprocessing import (
    create_dataset,
    moving_average_causal_filter,
    split_train_test_forecast_windows,
    find_peak_indices,
    create_features,
    normalise_dataset_multi_feature,
    select_features
)
from utils.dataset import SlowEarthquakeDataset
from utils.general_functions import set_seed, set_torch_device
from utils.nn_train import train_model_multi_feature, eval_model_on_test_set_multi_feature
from utils.plotting import plot_random_window, plot_random_test_window

# Set a random seed
SEED = 42
set_seed(SEED)

# Set the PyTorch device (GPU/cuda or CPU)
device = set_torch_device()

# If the notebook is being run on the JASMIN GPU cluster, select the second GPU (index = 1)
if socket.gethostname() == "gpuhost001.jc.rl.ac.uk":
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

No GPU available.


## Load Data

In [2]:
exp = "p4679"    # Set this to the name of the experiment you want to train on

if exp == "cascadia":
    column_name = "seg_avg"
else:
    column_name = "obs_shear_stress"

dataset = SlowEarthquakeDataset(exp)
df = pd.DataFrame(
    SlowEarthquakeDataset.convert_to_df(dataset, exp)[column_name].rename(
        "signal"
    )
)

if exp == "cascadia":
    df = df / 1e8  # Scale the slip potency signal to match magnitude of lab and sim data

df

,signal
0,5.091520
1,5.090652
2,5.089989
3,5.089492
4,5.088243
...,...
301716,4.979753
301717,4.979841
301718,4.980150
301719,4.979985


## Pre-process the data

In [3]:
# Define smoothing window and downsampling factor for each experiment (must be integers)
settings = {
    "cascadia": {"smoothing_window": 10, "downsampling_factor": 1},
    "p4679": {"smoothing_window": 20, "downsampling_factor": 13},
    "p4581": {"smoothing_window": 30, "downsampling_factor": 26},
    "b726": {"smoothing_window": 1, "downsampling_factor": 1},
    "b698": {"smoothing_window": 2, "downsampling_factor": 2},
    "i417": {"smoothing_window": 2, "downsampling_factor": 2},
    "sim_b726": {"smoothing_window": 1, "downsampling_factor": 1},
    "sim_b698": {"smoothing_window": 2, "downsampling_factor": 2},
    "sim_i417": {"smoothing_window": 2, "downsampling_factor": 2},
}

df_filtered = moving_average_causal_filter(df, **settings[exp])

df_filtered

,signal
0,5.067449
1,4.987701
2,4.912222
3,4.899803
4,4.897135
...,...
23203,4.977414
23204,4.977963
23205,4.978566
23206,4.979127


In [4]:
df_filtered.copy()

,signal
0,5.067449
1,4.987701
2,4.912222
3,4.899803
4,4.897135
...,...
23203,4.977414
23204,4.977963
23205,4.978566
23206,4.979127
